#Генерация известных потенциалов

В данном ноутбуке происходит генерация известных потенциала в приближении LDA с помощью функционала пакета pycsf - lda_x, lda_c_chachiyo и lda_c_pw_mod. Результат - директории с соответствующими .csv файлами с двумя колонками: x_0 (электронная плотность в электронах на кубический бор) и y (величина посчитанного с помощью программного пакета потенциала)

Сперва установим необходимые зависимости

In [1]:
!pip install pyscf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 11.5 MB/s eta 0:00:00


In [2]:
import pyscf
import pandas as pd
import numpy as np
np.random.seed(10)
import random
random.seed(10)
from typing import Tuple, Union, List, Dict, Callable, Any
from pyscf import gto, dft
from pyscf.dft import numint

/usr/local/lib/python3.10/dist-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


Для первого эксперимента сгенерируем электронную плотность в небольшом диапазоне. Однако, этого разброса значений хватьает, чтобы успешно считать многие квантовомеханические системы

In [ ]:
def generate_pseudopotentials(preudopotential):
  !mkdir "{preudopotential}"
  rho = np.concatenate([np.random.uniform(0, 200, 500), np.random.uniform(400, 600, 500), np.random.uniform(800, 1000, 500)])
  rho_test = np.concatenate([np.random.uniform(200, 400, 500), np.random.uniform(600, 800, 500)])
  exc, _, _, _ = dft.libxc.eval_xc(f"{preudopotential}", rho, spin=0)
  exc_test, _, _, _ = dft.libxc.eval_xc(f"{preudopotential}", rho_test, spin=0)
  pd.DataFrame({'x_0': rho, 'y': exc}).to_csv(f'{preudopotential}/{preudopotential}_first.csv', index=False)
  pd.DataFrame({'x_0': rho_test, 'y': exc_test}).to_csv(f'{preudopotential}/{preudopotential}_test_first.csv', index=False)

generate_pseudopotentials('lda_c_chachiyo')
generate_pseudopotentials('lda_x')
generate_pseudopotentials('lda_c_pw_mod')

Проверим корректность расчета потенциалов, исходя из физического смысла сгенерированных потенциалов (обменного lda_x и корреляционного lda_c_pw_mod)

In [ ]:
rho = np.concatenate([np.random.uniform(0, 100, 500), np.random.uniform(200, 300, 500), np.random.uniform(400, 500, 500)], axis=0)

exc_lda_x, _, _, _ = dft.libxc.eval_xc("lda_x", rho, spin=0)
exc_lda_c_pw_mod, _, _, _ = dft.libxc.eval_xc("lda_c_pw_mod", rho, spin=0)
exc_both, _, _, _ = dft.libxc.eval_xc("lda_x + lda_c_pw_mod", rho, spin=0)

print(set(exc_lda_x + exc_lda_c_pw_mod == exc_both))

{True}


Далее, проведем расчет электронной плотности атома ртути в базисе UGBS. Максимальное значение (или около него) будет верхней границей для второго эксперимента с моделями символьной регрессии

In [3]:
from pyscf import gto, dft
from pyscf.dft import numint

mol = gto.M(atom='Hg 0 0 0', basis='/content/ugbs.0.nw')

mf_orig = dft.RKS(mol)
mf_orig.xc = 'lda' #https://pyscf.org/user/dft.html
mf_orig.kernel()
dm = mf_orig.make_rdm1()

coords = mf_orig.grids.coords
weights = mf_orig.grids.weights
ao_value = numint.eval_ao(mol, coords, deriv=0)

rho = numint.eval_rho(mol, ao_value, dm, xctype='LDA')
print(rho.max(), rho.min(), rho.mean())

converged SCF energy = -18395.9178664542
364543.63191059907 1.2919395957932295e-11 7593.254119235434


Первое значение - 364543.63191059907, - и есть нужная нам верхняя граница. Сгенерируем электронную плотность в диапазоне от 0 до 10e+6. Точки будем брать не равномерно, а как степени 10, чтобы покрыть весь диапазон и не работать с сильно большим числом точек

In [4]:
def generate_pseudopotentials(preudopotential):
  !mkdir "{preudopotential}"
  rho = 10**np.concatenate([np.random.uniform(-1, 1, 1000), np.random.uniform(3, 5, 1000)])
  rho_test = 10**np.concatenate([np.random.uniform(-3, -1, 1000), np.random.uniform(1, 3, 1000), np.random.uniform(5, 6, 1000)])
  exc, _, _, _ = dft.libxc.eval_xc(f"{preudopotential}", rho, spin=0)
  exc_test, _, _, _ = dft.libxc.eval_xc(f"{preudopotential}", rho_test, spin=0)
  pd.DataFrame({'x_0': rho, 'y': exc}).to_csv(f'{preudopotential}/{preudopotential}_second.csv', index=False)
  pd.DataFrame({'x_0': rho_test, 'y': exc_test}).to_csv(f'{preudopotential}/{preudopotential}_test_second.csv', index=False)

generate_pseudopotentials('lda_c_chachiyo')
generate_pseudopotentials('lda_x')
generate_pseudopotentials('lda_c_pw_mod')

mkdir: cannot create directory ‘lda_c_chachiyo’: File exists
mkdir: cannot create directory ‘lda_x’: File exists
mkdir: cannot create directory ‘lda_x_1d_exponential’: File exists
mkdir: cannot create directory ‘lda_c_1d_csc’: File exists
mkdir: cannot create directory ‘lda_c_1d_loos’: File exists
mkdir: cannot create directory ‘lda_c_pw_mod’: File exists
